In [ ]:
!pip install -U google-generativeai

In [53]:
import google.generativeai as genai
from collections import Counter
from IPython.display import Markdown

In [3]:
# Set up Gemini-Pro API key
genai.configure(api_key="gemini-key")


The set_gemini_params function that allows to configure the Gemini-2.0-flash model with customizable parameters:


Temperature: Controls randomness (lower = deterministic, higher = diverse outputs).

Top_p: Nucleus sampling, controlling probability mass selection.

Frequency & Presence Penalty: Modify token repetition behavior.

In [4]:
def set_gemini_params(
    model='gemini-2.0-flash',
    temperature=0.7,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """Set Gemini-2.0-flash parameters."""
    params = {
        "model": model,
        "temperature": temperature,
        "top_p": top_p,
        "frequency_penalty": frequency_penalty,
        "presence_penalty": presence_penalty,
    }
    return params

In [5]:
def get_completion(params, prompt):
    """Get completion from Gemini-2.0-flash API."""
    try:
        model_name = params["model"]
        model = genai.GenerativeModel(model_name)
        response = model.generate_content(
            prompt,
            generation_config={
                "temperature": params["temperature"],
                "top_p": params["top_p"],
            },
        )
        return response.text if hasattr(response, "text") else "No response received."
    except Exception as e:
        return f"Error: {e}"


# 1. Few Shot Prompting



Purpose: Provides examples to guide the model’s response style.

Uses a low temperature (0.3) to ensure accuracy and consistency.

It responds translation task.

In [33]:
def few_shot():
    """Few-shot prompting """
    instructions = "Translate the following English sentences to French."
    context = "Here are some examples:\n1. English: Hello, how are you? French: Bonjour, comment ça va?\n2. English: I love programming. French: J'adore la programmation."
    input_data = "English: The weather is nice today."
    output_indicator = "French:"
    prompt = f"{instructions}\n{context}\n{input_data}\n{output_indicator}"


    params = set_gemini_params(temperature=0.3)  # Low temperature for deterministic output
    response = get_completion(params, prompt)
    display(Markdown(response))


In [17]:
if __name__ == "__main__":
    few_shot()

Il fait beau aujourd'hui.



# 2. Chain of Thought

Zero-shot Chain-of-Thought (CoT) Prompting

Purpose: Encourages step-by-step reasoning without examples.

* Works well for simple logical problems.

In [29]:
def zero_shot_cot():
    """Zero-shot chain-of-thought example."""
    instructions = "Solve the following math problem step by step."
    context = ""
    input_data = "If a car travels 60 miles in 1 hour, how far will it travel in 3 hours?"
    output_indicator = "Let's think step by step:"
    prompt = f"{instructions}\n{context}\n{input_data}\n{output_indicator}"

    params = set_gemini_params(temperature=0.3)  # Focus on logical consistency rather than unnecessary
    response = get_completion(params, prompt)
    display(Markdown(response))

In [30]:
if __name__ == "__main__":
    zero_shot_cot()

1. **Find the speed:** The car travels 60 miles in 1 hour, so its speed is 60 miles per hour (mph).

2. **Calculate the distance:** To find the distance traveled in 3 hours, multiply the speed by the time: 60 mph * 3 hours = 180 miles.

**Answer:** The car will travel 180 miles in 3 hours.


Few-shot Chain-of-Thought (CoT) Prompting

Purpose: Enhances reasoning by using prior structured examples.

In [31]:
def few_shot_cot():
    """Few-shot chain-of-thought example."""
    instructions = "Solve the following math problem step by step."
    context = """
Here are some examples:
1. Problem: If a car travels 60 miles in 1 hour, how far will it travel in 3 hours?
   Solution: Let's think step by step:
   - The car travels 60 miles in 1 hour.
   - To find the distance traveled in 3 hours, multiply the speed by the time.
   - 60 miles/hour * 3 hours = 180 miles.
   Answer: 180 miles.

2. Problem: If a train travels 120 kilometers in 2 hours, how far will it travel in 5 hours?
   Solution: Let's think step by step:
   - The train travels 120 kilometers in 2 hours.
   - First, find the speed: 120 km / 2 hours = 60 km/hour.
   - To find the distance traveled in 5 hours, multiply the speed by the time.
   - 60 km/hour * 5 hours = 300 kilometers.
   Answer: 300 kilometers.
"""
    input_data = "Problem: If a cyclist travels 30 kilometers in 1 hour, how far will they travel in 4 hours?"
    output_indicator = "Solution: "
    prompt = f"{instructions}\n{context}\n{input_data}\n{output_indicator}"

    params = set_gemini_params(temperature=0.3)  # Low temperature for deterministic output
    response = get_completion(params, prompt)
    display(Markdown(response))

In [32]:
if __name__ == "__main__":
    few_shot_cot()

- The cyclist travels 30 kilometers in 1 hour.
- To find the distance traveled in 4 hours, multiply the speed by the time.
- 30 kilometers/hour * 4 hours = 120 kilometers.
Answer: 120 kilometers.


# 3. Self-Consistency


Purpose: Runs multiple completions and selects the most frequent output.

* Temperature kept at 0.3 to minimize randomness.

* Uses Counter to identify the most common response.

In [54]:
def self_consistency():
    """Self consistency """
    instructions = "Answer the following question with a single, direct answer."
    context = ""
    input_data = "Question: What is the most poisonous insect?"
    output_indicator = "Answer:"
    prompt = f"{instructions}\n{context}\n{input_data}\n{output_indicator}"

    params = set_gemini_params(temperature=0.3)  # Lower temperature for focused output

    # Generate multiple responses
    responses = []
    for _ in range(5):  # Generate 5 responses
        response = get_completion(params, prompt)
        responses.append(response)

    # Determine the most consistent answer
    most_common_answer = Counter(responses).most_common(1)[0][0]

    display(Markdown("### Generated Responses:"))
    for i, response in enumerate(responses, 1):
        display(Markdown(f"**Response {i}:** {response}"))

    # Display the most consistent answer
    display(Markdown(f"\n**Most Consistent Answer:** {most_common_answer}"))


In [55]:
if __name__ == "__main__":
  self_consistency()

### Generated Responses:

**Response 1:** The most poisonous insect is debated, but the **poison dart frog** is often cited as the most poisonous animal, and while not an insect, its poison is derived from insects in its diet.


**Response 2:** The most poisonous insect is the **blister beetle**.


**Response 3:** The most poisonous insect is debated, but the **blister beetle** is often cited due to its cantharidin content.


**Response 4:** The most poisonous insect is generally considered to be the **blister beetle**.


**Response 5:** The most poisonous insect is generally considered to be the **blister beetle**.



**Most Consistent Answer:** The most poisonous insect is generally considered to be the **blister beetle**.


# 4. Generate Knowledge

Purpose: Generates knowledge first, then uses it to answer a question.


*   Multi-step response generation.






In [43]:
def generate_knowledge_dual_prompt():
    """Generate knowledge about AI in healthcare and use it to answer a question."""
    # Step 1: Generate short background knowledge about Cloud Computing in Inventory Management
    instructions = "Generate concise background knowledge about the role of Cloud Computing in inventory management."
    context = ""
    input_data = "Topic: Cloud Computing in Inventory Management"
    output_indicator = "Knowledge:"
    prompt = f"{instructions}\n{context}\n{input_data}\n{output_indicator}"

    params = set_gemini_params(temperature=0.5)  # Moderate temperature since the goal is exploratory content
    knowledge = get_completion(params, prompt)

    # Step 2: Use the generated knowledge to answer a specific question
    instructions = "Answer the question using the provided knowledge."
    context = f"Knowledge: {knowledge}"
    input_data = "Question: What are the key advantages of Cloud Computing in Inventory management?"
    output_indicator = "Answer:"
    prompt = f"{instructions}\n{context}\n{input_data}\n{output_indicator}"

    response = get_completion(params, prompt)

    print("Background Knowledge")
    display(Markdown(knowledge))
    print("Answer")
    display(Markdown(response))

In [44]:
if __name__ == "__main__":
    generate_knowledge_dual_prompt()

Background Knowledge


Cloud computing offers scalable, accessible, and cost-effective solutions for inventory management. It enables real-time visibility across the supply chain, improves forecasting accuracy, streamlines warehouse operations, and facilitates better decision-making through data analytics, all without the need for extensive on-premise infrastructure.


Answer


The key advantages of Cloud Computing in inventory management are:

*   **Scalability:** Easily adjust resources to meet changing inventory needs.
*   **Accessibility:** Access inventory data from anywhere with an internet connection.
*   **Cost-effectiveness:** Reduces the need for expensive on-premise infrastructure.
*   **Real-time visibility:** Provides up-to-date information across the supply chain.
*   **Improved forecasting accuracy:** Enables better prediction of future demand.
*   **Streamlined warehouse operations:** Optimizes processes within the warehouse.
*   **Better decision-making:** Facilitates informed choices through data analytics.


#5. Program-Aided

Purpose: Debugs code by prompting the model for corrections.

* Fixes logical and syntax errors in Python.

In [45]:
def program_aided():
    """Program-aided language model for debugging Python code."""
    instructions = "Find and fix the bug in the following Python function."
    context = "def divide(a, b):\n    return a / b\n\nprint(divide(5, 0))"
    output_indicator = "Corrected code:"
    prompt = f"{instructions}\n{context}\n{output_indicator}"

    params = set_gemini_params(temperature=0.3)
    response = get_completion(params, prompt)
    display(Markdown(response))

In [46]:
if __name__ == "__main__":
    program_aided()

```python
def divide(a, b):
    if b == 0:
        return "Cannot divide by zero"
    return a / b

print(divide(5, 0))
```

**Reasoning:**

The original code does not handle the case where `b` is zero, which will cause a `ZeroDivisionError`. The corrected code adds a check to see if `b` is zero. If it is, it returns an error message. Otherwise, it performs the division as before. This prevents the program from crashing and provides a more informative response to the user.
